# Import

In [84]:
# import libraries
import numpy as np 
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer


In [50]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\princ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
data = pd.read_csv("./data/flipkart_com-ecommerce_sample_1050.csv")
data.head(3)

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-mul...,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",CRNEG7BKMFFYHQ8Z,1899.0,899.0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,Key Features of Elegance Polyester Multicolor ...,No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
1,7b72c92c2f6c40268628ec5f14c6d590,2016-04-30 03:22:56 +0000,http://www.flipkart.com/sathiyas-cotton-bath-t...,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEGFZHGBXPHZUH,600.0,449.0,7b72c92c2f6c40268628ec5f14c6d590.jpg,False,Specifications of Sathiyas Cotton Bath Towel (...,No rating available,No rating available,Sathiyas,"{""product_specification""=>[{""key""=>""Machine Wa..."
2,64d5d4a258243731dc7bbb1eef49ad74,2016-04-30 03:22:56 +0000,http://www.flipkart.com/eurospa-cotton-terry-f...,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEG6SHXTDB2A2Y,NaN,NaN,64d5d4a258243731dc7bbb1eef49ad74.jpg,False,Key Features of Eurospa Cotton Terry Face Towe...,No rating available,No rating available,Eurospa,"{""product_specification""=>[{""key""=>""Material"",..."


In [8]:
corpus = data[['description']]
corpus.head(3)

,description
0,Key Features of Elegance Polyester Multicolor ...
1,Specifications of Sathiyas Cotton Bath Towel (...
2,Key Features of Eurospa Cotton Terry Face Towe...


In [9]:
corpus.shape

(1050, 1)

# Data preprocessing

##  Récupération des catégories principales

In [10]:
categories = data[['product_category_tree']] 
categories

,product_category_tree
0,"[""Home Furnishing >> Curtains & Accessories >>..."
1,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T..."
2,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T..."
3,"[""Home Furnishing >> Bed Linen >> Bedsheets >>..."
4,"[""Home Furnishing >> Bed Linen >> Bedsheets >>..."
...,...
1045,"[""Baby Care >> Baby & Kids Gifts >> Stickers >..."
1046,"[""Baby Care >> Baby & Kids Gifts >> Stickers >..."
1047,"[""Baby Care >> Baby & Kids Gifts >> Stickers >..."
1048,"[""Baby Care >> Baby & Kids Gifts >> Stickers >..."


In [11]:
categories.iloc[1]

product_category_tree    ["Baby Care >> Baby Bath & Skin >> Baby Bath T...
Name: 1, dtype: object

In [23]:
main_category['main_cat'] = categories.product_category_tree.str.split(">>",expand=True,)[0]
main_category

0                                          ["Home Furnishing 
1                                                ["Baby Care 
2                                                ["Baby Care 
3                                          ["Home Furnishing 
4                                          ["Home Furnishing 
                                  ...                        
1046                                             ["Baby Care 
1047                                             ["Baby Care 
1048                                             ["Baby Care 
1049                                             ["Baby Care 
main_cat    0       ["Home Furnishing 
1             ["Bab...
Name: 0, Length: 1051, dtype: object

In [34]:
main_category['main_cat'] = main_category['main_cat'].str.replace(r'"', '')
main_category['main_cat'] = main_category['main_cat'].str.replace(r'[', '')
main_category['main_cat'] = main_category['main_cat'].str.replace(r']', '')

<ipython-input-34-98ee0773f790>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  main_category['main_cat'] = main_category['main_cat'].str.replace(r'[', '')
<ipython-input-34-98ee0773f790>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  main_category['main_cat'] = main_category['main_cat'].str.replace(r']', '')


In [35]:
main_category['main_cat'].unique()

array(['Home Furnishing ', 'Baby Care ', 'Watches ',
       'Home Decor & Festive Needs ', 'Kitchen & Dining ',
       'Beauty and Personal Care ', 'Computers '], dtype=object)

## Lowercasing

In [37]:
corpus

,description
0,Key Features of Elegance Polyester Multicolor ...
1,Specifications of Sathiyas Cotton Bath Towel (...
2,Key Features of Eurospa Cotton Terry Face Towe...
3,Key Features of SANTOSH ROYAL FASHION Cotton P...
4,Key Features of Jaipur Print Cotton Floral Kin...
...,...
1045,Oren Empower Extra Large Self Adhesive Sticker...
1046,Wallmantra Large Vinyl Sticker Sticker (Pack o...
1047,Buy Uberlyfe Extra Large Pigmented Polyvinyl F...
1048,Buy Wallmantra Medium Vinyl Sticker Sticker fo...


In [40]:
corpus['description'] = corpus['description'].str.lower()
corpus

<ipython-input-40-032e26959556>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus['description'] = corpus['description'].str.lower()


,description
0,key features of elegance polyester multicolor ...
1,specifications of sathiyas cotton bath towel (...
2,key features of eurospa cotton terry face towe...
3,key features of santosh royal fashion cotton p...
4,key features of jaipur print cotton floral kin...
...,...
1045,oren empower extra large self adhesive sticker...
1046,wallmantra large vinyl sticker sticker (pack o...
1047,buy uberlyfe extra large pigmented polyvinyl f...
1048,buy wallmantra medium vinyl sticker sticker fo...


## Stop words

In [47]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [63]:
str_corpus = corpus.description.str.cat(sep=' ')

In [65]:
stop_words = set(stopwords.words('english'))
 
word_tokens = word_tokenize(str_corpus)
 
filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
 
filtered_sentence = []
 
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)

In [69]:
filtered_sentence

['key',
 'features',
 'elegance',
 'polyester',
 'multicolor',
 'abstract',
 'eyelet',
 'door',
 'curtain',
 'floral',
 'curtain',
 ',',
 'elegance',
 'polyester',
 'multicolor',
 'abstract',
 'eyelet',
 'door',
 'curtain',
 '(',
 '213',
 'cm',
 'height',
 ',',
 'pack',
 '2',
 ')',
 'price',
 ':',
 'rs',
 '.',
 '899',
 'curtain',
 'enhances',
 'look',
 'interiors.this',
 'curtain',
 'made',
 '100',
 '%',
 'high',
 'quality',
 'polyester',
 'fabric.it',
 'features',
 'eyelet',
 'style',
 'stitch',
 'metal',
 'ring.it',
 'makes',
 'room',
 'environment',
 'romantic',
 'loving.this',
 'curtain',
 'ant-',
 'wrinkle',
 'anti',
 'shrinkage',
 'elegant',
 'apparance.give',
 'home',
 'bright',
 'modernistic',
 'appeal',
 'designs',
 '.',
 'surreal',
 'attention',
 'sure',
 'steal',
 'hearts',
 '.',
 'contemporary',
 'eyelet',
 'valance',
 'curtains',
 'slide',
 'smoothly',
 'draw',
 'apart',
 'first',
 'thing',
 'morning',
 'welcome',
 'bright',
 'sun',
 'rays',
 'want',
 'wish',
 'good',
 'mo

In [67]:
len(filtered_sentence)

72368

## Lemmatisation  

In [78]:
ps = PorterStemmer()

stemmed_sentences = []
for w in filtered_sentence:
    print(w, " : ", ps.stem(w))
    stemmed_sentences.append(ps.stem(w))

key  :  key
features  :  featur
elegance  :  eleg
polyester  :  polyest
multicolor  :  multicolor
abstract  :  abstract
eyelet  :  eyelet
door  :  door
curtain  :  curtain
floral  :  floral
curtain  :  curtain
,  :  ,
elegance  :  eleg
polyester  :  polyest
multicolor  :  multicolor
abstract  :  abstract
eyelet  :  eyelet
door  :  door
curtain  :  curtain
(  :  (
213  :  213
cm  :  cm
height  :  height
,  :  ,
pack  :  pack
2  :  2
)  :  )
price  :  price
:  :  :
rs  :  rs
.  :  .
899  :  899
curtain  :  curtain
enhances  :  enhanc
look  :  look
interiors.this  :  interiors.thi
curtain  :  curtain
made  :  made
100  :  100
%  :  %
high  :  high
quality  :  qualiti
polyester  :  polyest
fabric.it  :  fabric.it
features  :  featur
eyelet  :  eyelet
style  :  style
stitch  :  stitch
metal  :  metal
ring.it  :  ring.it
makes  :  make
room  :  room
environment  :  environ
romantic  :  romant
loving.this  :  loving.thi
curtain  :  curtain
ant-  :  ant-
wrinkle  :  wrinkl
anti  :  anti
shrink

sheet  :  sheet
depth  :  depth
0.2  :  0.2
cm  :  cm
flat  :  flat
sheet  :  sheet
length  :  length
104  :  104
inch  :  inch
/  :  /
265  :  265
cm  :  cm
box  :  box
number  :  number
contents  :  content
sales  :  sale
package  :  packag
1  :  1
sales  :  sale
package  :  packag
1  :  1
bed  :  bed
sheet  :  sheet
2  :  2
pillow  :  pillow
cover  :  cover
maserati  :  maserati
time  :  time
r8851116001  :  r8851116001
analog  :  analog
watch  :  watch
-  :  -
boys  :  boy
-  :  -
buy  :  buy
maserati  :  maserati
time  :  time
r8851116001  :  r8851116001
analog  :  analog
watch  :  watch
-  :  -
boys  :  boy
r8851116001  :  r8851116001
online  :  onlin
rs.24400  :  rs.24400
india  :  india
flipkart.com  :  flipkart.com
.  :  .
-  :  -
great  :  great
discounts  :  discount
,  :  ,
genuine  :  genuin
products  :  product
,  :  ,
30  :  30
day  :  day
replacement  :  replac
guarantee  :  guarante
,  :  ,
free  :  free
shipping  :  ship
.  :  .
cash  :  cash
delivery  :  deliveri
!  

watch  :  watch
-  :  -
boys  :  boy
-  :  -
buy  :  buy
jack  :  jack
klein  :  klein
blackled  :  blackl
digital  :  digit
watch  :  watch
-  :  -
boys  :  boy
blackled  :  blackl
online  :  onlin
rs.150  :  rs.150
india  :  india
flipkart.com  :  flipkart.com
.  :  .
-  :  -
great  :  great
discounts  :  discount
,  :  ,
genuine  :  genuin
products  :  product
,  :  ,
30  :  30
day  :  day
replacement  :  replac
guarantee  :  guarante
,  :  ,
free  :  free
shipping  :  ship
.  :  .
cash  :  cash
delivery  :  deliveri
!  :  !
north  :  north
moon  :  moon
iw-005-fk  :  iw-005-fk
silicone  :  silicon
ion  :  ion
digital  :  digit
watch  :  watch
-  :  -
boys  :  boy
,  :  ,
girls  :  girl
,  :  ,
women  :  women
-  :  -
buy  :  buy
north  :  north
moon  :  moon
iw-005-fk  :  iw-005-fk
silicone  :  silicon
ion  :  ion
digital  :  digit
watch  :  watch
-  :  -
boys  :  boy
,  :  ,
girls  :  girl
,  :  ,
women  :  women
iw-005-fk  :  iw-005-fk
online  :  onlin
rs.150  :  rs.150
india  : 

silicone  :  silicon
,  :  ,
strap  :  strap
width:20mm  :  width:20mm
,  :  ,
water  :  water
resistant  :  resist
,  :  ,
night  :  night
display  :  display
cold  :  cold
backlight  :  backlight
,  :  ,
daily  :  daili
alarm  :  alarm
,  :  ,
hour  :  hour
minute  :  minut
second  :  second
,  :  ,
date  :  date
week  :  week
display  :  display
,  :  ,
12/24  :  12/24
hour  :  hour
format  :  format
,  :  ,
attractive  :  attract
brand  :  brand
packaging-  :  packaging-
tuff  :  tuff
plastic  :  plastic
box  :  box
,  :  ,
note  :  note
:  :  :
case  :  case
non-receipt  :  non-receipt
warranty  :  warranti
card  :  card
,  :  ,
please  :  pleas
intimate  :  intim
us  :  us
within  :  within
48  :  48
hours  :  hour
,  :  ,
disclaimer  :  disclaim
:  :  :
product  :  product
color  :  color
may  :  may
slightly  :  slightli
vary  :  vari
due  :  due
photographic  :  photograph
lighting  :  light
sources  :  sourc
monitor  :  monitor
settings  :  set
(  :  (
image  :  imag
enlarged

copper  :  copper
alloy  :  alloy
surfaces  :  surfac
(  :  (
65  :  65
%  :  %
greater  :  greater
copper  :  copper
content  :  content
)  :  )
eliminated  :  elimin
within  :  within
2  :  2
hours  :  hour
exposure  :  exposur
.  :  .
according  :  accord
research  :  research
southampton  :  southampton
university  :  univers
u.k.  :  u.k.
,  :  ,
mrsa  :  mrsa
microbes  :  microb
remain  :  remain
alive  :  aliv
stainless  :  stainless
steel  :  steel
surfaces  :  surfac
three  :  three
days  :  day
,  :  ,
whereas  :  wherea
microbes  :  microb
copper  :  copper
surface  :  surfac
eliminated  :  elimin
within  :  within
90  :  90
minutes  :  minut
.  :  .
copper  :  copper
serving  :  serv
ware  :  ware
preferred  :  prefer
durable  :  durabl
excellent  :  excel
characteristics  :  characterist
.  :  .
ageless  :  ageless
use  :  use
copper  :  copper
bears  :  bear
testimony  :  testimoni
myriad  :  myriad
virtues  :  virtu
everlasting  :  everlast
appeal  :  appeal
.  :  .
ther

...

,  :  ,
lawman  :  lawman
pg3  :  pg3
striker  :  striker
,  :  ,
,  :  ,
winger  :  winger
,  :  ,
contender  :  contend
deodorant  :  deodor
spray  :  spray
-  :  -
men  :  men
(  :  (
210  :  210
ml  :  ml
)  :  )
price  :  price
:  :  :
rs  :  rs
.  :  .
447  :  447
lawman  :  lawman
deodorant  :  deodor
combo  :  combo
men  :  men
,  :  ,
specifications  :  specif
lawman  :  lawman
pg3  :  pg3
striker  :  striker
,  :  ,
,  :  ,
winger  :  winger
,  :  ,
contender  :  contend
deodorant  :  deodor
spray  :  spray
-  :  -
men  :  men
(  :  (
210  :  210
ml  :  ml
)  :  )
fragrance  :  fragranc
traits  :  trait
fragrance  :  fragranc
classification  :  classif
deodorant  :  deodor
spray  :  spray
quantity  :  quantiti
210  :  210
ml  :  ml
ideal  :  ideal
men  :  men
box  :  box
3  :  3
deodorant  :  deodor
key  :  key
features  :  featur
toons  :  toon
printed  :  print
baby  :  babi
boy  :  boy
's  :  's
round  :  round
neck  :  neck
t-shirt  :  t-shirt
stylish  :  stylish
,  :  ,


patta  :  patta
model  :  model
id  :  id
ac-558  :  ac-558
color  :  color
blue  :  blue
dimensions  :  dimens
length  :  length
274  :  274
cm  :  cm
body  :  bodi
&  :  &
design  :  design
material  :  materi
polyester  :  polyest
box  :  box
number  :  number
contents  :  content
sales  :  sale
package  :  packag
pack  :  pack
3  :  3
key  :  key
features  :  featur
print  :  print
shapes  :  shape
roronoa  :  roronoa
zora  :  zora
laptop  :  laptop
skin  :  skin
mouse  :  mous
pad  :  pad
combo  :  combo
set  :  set
glue  :  glue
free  :  free
removal  :  remov
15.6  :  15.6
inches  :  inch
laptop  :  laptop
skin  :  skin
easy  :  easi
apply  :  appli
&  :  &
scratch  :  scratch
resistant  :  resist
lamination  :  lamin
,  :  ,
print  :  print
shapes  :  shape
roronoa  :  roronoa
zora  :  zora
laptop  :  laptop
skin  :  skin
mouse  :  mous
pad  :  pad
combo  :  combo
set  :  set
(  :  (
multicolor  :  multicolor
)  :  )
price  :  price
:  :  :
rs  :  rs
.  :  .
329  :  329
print  

print  :  print
shapes  :  shape
mary  :  mari
max  :  max
cartoon  :  cartoon
laptop  :  laptop
skin  :  skin
mouse  :  mous
pad  :  pad
combo  :  combo
set  :  set
(  :  (
multicolor  :  multicolor
)  :  )
price  :  price
:  :  :
rs  :  rs
.  :  .
329  :  329
print  :  print
shapes  :  shape
laptop  :  laptop
skin  :  skin
made  :  made
high  :  high
quality  :  qualiti
vinyl  :  vinyl
sheet  :  sheet
compatible  :  compat
laptops  :  laptop
screen  :  screen
sizes  :  size
ranging  :  rang
14.1  :  14.1
inches  :  inch
15.6  :  15.6
inches  :  inch
.  :  .
trimning  :  trimn
skin  :  skin
would  :  would
required  :  requir
per  :  per
laptop  :  laptop
size  :  size
.  :  .
skins  :  skin
easy  :  easi
install  :  instal
&  :  &
remove  :  remov
without  :  without
leaving  :  leav
gum  :  gum
residue  :  residu
,  :  ,
specifications  :  specif
print  :  print
shapes  :  shape
mary  :  mari
max  :  max
cartoon  :  cartoon
laptop  :  laptop
skin  :  skin
mouse  :  mous
pad  :  pad


viscous  :  viscou
fabric  :  fabric
.  :  .
v-shaped  :  v-shape
neck  :  neck
top  :  top
&  :  &
trouser  :  trouser
.  :  .
designed  :  design
way  :  way
give  :  give
complete  :  complet
comfort  :  comfort
feel  :  feel
operate  :  oper
.  :  .
available  :  avail
,  :  ,
,  :  ,
l  :  l
&  :  &
xl  :  xl
sizes  :  size
available  :  avail
different  :  differ
colours  :  colour
match  :  match
hospital/nursing  :  hospital/nurs
home/clinic  :  home/clin
's  :  's
logo  :  logo
theme  :  theme
best  :  best
.  :  .
also  :  also
name  :  name
customized  :  custom
aprons  :  apron
&  :  &
complete  :  complet
staff  :  staff
uniforms  :  uniform
males  :  male
&  :  &
females.  :  females.
,  :  ,
specifications  :  specif
surgical  :  surgic
e  :  e
sstudio  :  sstudio
s0004  :  s0004
gown  :  gown
,  :  ,
pant  :  pant
hospital  :  hospit
scrub  :  scrub
(  :  (
pink  :  pink
)  :  )
general  :  gener
brand  :  brand
surgical  :  surgic
e  :  e
sstudio  :  sstudio
brand  :  

)  :  )
simply  :  simpli
peel  :  peel
pre-cut  :  pre-cut
pieces  :  piec
wall  :  wall
stickers  :  sticker
backing  :  back
paper  :  paper
apply  :  appli
desired  :  desir
area  :  area
.  :  .
note  :  note
:  :  :
1.  :  1.
freshly  :  freshli
painted  :  paint
lacquered  :  lacquer
surfaces  :  surfac
must  :  must
allowed  :  allow
completely  :  complet
cure  :  cure
decal  :  decal
applied  :  appli
.  :  .
recommend  :  recommend
waiting  :  wait
minimum  :  minimum
2-3  :  2-3
weeks  :  week
.  :  .
2.  :  2.
discourage  :  discourag
installation  :  instal
certain  :  certain
surfaces  :  surfac
brick  :  brick
,  :  ,
stucco  :  stucco
,  :  ,
concrete  :  concret
block  :  block
,  :  ,
suede  :  sued
paints  :  paint
,  :  ,
sand  :  sand
paints  :  paint
.  :  .
3.  :  3.
apply  :  appli
wet  :  wet
walls  :  wall
3  :  3
)  :  )
press  :  press
firmly  :  firmli
squeeze  :  squeez
air  :  air
bubbles  :  bubbl
.  :  .
note  :  note
:  :  :
helpful  :  help
fun  :  f

In [79]:
stemmed_sentences

['key',
 'featur',
 'eleg',
 'polyest',
 'multicolor',
 'abstract',
 'eyelet',
 'door',
 'curtain',
 'floral',
 'curtain',
 ',',
 'eleg',
 'polyest',
 'multicolor',
 'abstract',
 'eyelet',
 'door',
 'curtain',
 '(',
 '213',
 'cm',
 'height',
 ',',
 'pack',
 '2',
 ')',
 'price',
 ':',
 'rs',
 '.',
 '899',
 'curtain',
 'enhanc',
 'look',
 'interiors.thi',
 'curtain',
 'made',
 '100',
 '%',
 'high',
 'qualiti',
 'polyest',
 'fabric.it',
 'featur',
 'eyelet',
 'style',
 'stitch',
 'metal',
 'ring.it',
 'make',
 'room',
 'environ',
 'romant',
 'loving.thi',
 'curtain',
 'ant-',
 'wrinkl',
 'anti',
 'shrinkag',
 'eleg',
 'apparance.g',
 'home',
 'bright',
 'modernist',
 'appeal',
 'design',
 '.',
 'surreal',
 'attent',
 'sure',
 'steal',
 'heart',
 '.',
 'contemporari',
 'eyelet',
 'valanc',
 'curtain',
 'slide',
 'smoothli',
 'draw',
 'apart',
 'first',
 'thing',
 'morn',
 'welcom',
 'bright',
 'sun',
 'ray',
 'want',
 'wish',
 'good',
 'morn',
 'whole',
 'world',
 'draw',
 'close',
 'even'

## Bags of words

### Bigrams

In [100]:
list(nltk.bigrams(stemmed_sentences))

[('key', 'featur'),
 ('featur', 'eleg'),
 ('eleg', 'polyest'),
 ('polyest', 'multicolor'),
 ('multicolor', 'abstract'),
 ('abstract', 'eyelet'),
 ('eyelet', 'door'),
 ('door', 'curtain'),
 ('curtain', 'floral'),
 ('floral', 'curtain'),
 ('curtain', ','),
 (',', 'eleg'),
 ('eleg', 'polyest'),
 ('polyest', 'multicolor'),
 ('multicolor', 'abstract'),
 ('abstract', 'eyelet'),
 ('eyelet', 'door'),
 ('door', 'curtain'),
 ('curtain', '('),
 ('(', '213'),
 ('213', 'cm'),
 ('cm', 'height'),
 ('height', ','),
 (',', 'pack'),
 ('pack', '2'),
 ('2', ')'),
 (')', 'price'),
 ('price', ':'),
 (':', 'rs'),
 ('rs', '.'),
 ('.', '899'),
 ('899', 'curtain'),
 ('curtain', 'enhanc'),
 ('enhanc', 'look'),
 ('look', 'interiors.thi'),
 ('interiors.thi', 'curtain'),
 ('curtain', 'made'),
 ('made', '100'),
 ('100', '%'),
 ('%', 'high'),
 ('high', 'qualiti'),
 ('qualiti', 'polyest'),
 ('polyest', 'fabric.it'),
 ('fabric.it', 'featur'),
 ('featur', 'eyelet'),
 ('eyelet', 'style'),
 ('style', 'stitch'),
 ('stitch'

### Tfidf

In [110]:
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(corpus['description'])
# summarize
print(sorted(vectorizer.vocabulary_))
# encode document
vector = vectorizer.transform([corpus['description'][0]])

['00', '000', '001', '0021', '004', '005', '006', '0083', '01', '011', '01433cmgy', '01727lpln', '01741lpln', '01784bmly', '02', '03', '03918cmly', '04', '04615cmgy', '050', '052', '05712lmly', '05tg', '06', '063', '06362cmgy', '064', '069', '07', '07034lmli', '074', '08', '085', '09', '09321cmgy', '099', '10', '100', '1000', '1001', '10024982', '1008ax', '101', '1010', '1014tx', '1015', '1015nr', '101b0101', '1020el', '1033', '1038', '104', '1042', '1045', '1049', '105key', '107', '1071', '1075', '107750', '108', '1080', '1085', '10890', '1092', '1099', '109b0109', '10btsc2012gb', '11', '110', '1100', '11000026', '111', '112', '1132', '1141ym22', '1142301', '1143', '1148', '1149', '1150', '1153', '1155', '1170', '1183', '119', '1195', '1199', '11n', '11oz', '12', '120', '1200', '12011ppgw', '121', '1216', '122', '1234', '12399', '1240sg', '1241ea', '125', '125g', '125prog', '126', '129', '1290', '1295', '1299', '12v', '12x12', '12x18', '13', '130', '132', '1333', '13400', '1345', '134

In [107]:
print(vectorizer.idf_)

[7.26435019 7.26435019 6.85888508 ... 7.26435019 7.26435019 6.57120301]


In [111]:
print(vector.shape)

(1, 6053)


In [112]:
print(vector.toarray())

[[0. 0. 0. ... 0. 0. 0.]]
